In [210]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent

# Data mining

### In this notebook I am mining data on open listings of houses for sale in Sochi, Russia from [cian.ru](https://sochi.cian.ru/kupit-dom/) for July 2024.

 Cian didn't want to share the data (they sell it for 150+ thousand rubles) and gave a `403 Forbidden` error on common http requests, so I used `Selenium` and a `fake_useragent`

In [211]:
import numpy as np
from tqdm import tqdm

#### This function I will use to generate random UserAgent and launch chrome browser with it

In [212]:
def get_random_useragent_driver():
    ua = UserAgent()
    user_agent = ua.random
    
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument(f"user-agent={user_agent}")
    
    s = Service("D:/chromedriver_win32/chromedriver.exe")
    driver = webdriver.Chrome(service=s, options=chrome_options)
    return driver

In [213]:
driver = get_random_useragent_driver()

#### Next code block was used to parsing links to specific home listings for sale

In [214]:

highlevel_links = ["https://sochi.cian.ru/kupit-dom/"] + [f"https://krasnodar.cian.ru/cat.php/?deal_type=sale&engine_version=2&object_type%5B0%5D=1&offer_type=suburban&p={i+1}&region=4998" for i in range(56)]

#print(highlevel_links)

house_cards_links = []
for i, link in tqdm(enumerate(highlevel_links)):
    driver.get(link)
    # Find all <a> tags on the webpage
    all_lowlevel_links = [link.get_attribute('href') for link in driver.find_elements(By.TAG_NAME, 'a')]
    sales_lowlevel_links = [link for link in all_lowlevel_links if "/sale/suburban/" in link]

    # Extract and print the href attribute for each <a> tag
    if i%10==0:
        print("Step", i, "; All house cards link count:", len(house_cards_links))
    house_cards_links.extend(sales_lowlevel_links)

1it [00:10, 10.01s/it]

Step 0 ; All house cards link count: 0


11it [01:04,  5.42s/it]

Step 10 ; All house cards link count: 1400


21it [01:52,  4.29s/it]

...

41it [03:07,  3.63s/it]

Step 40 ; All house cards link count: 5585


51it [03:43,  3.71s/it]

Step 50 ; All house cards link count: 6975


57it [04:12,  4.42s/it]


In [314]:
house_cards_links = list(set(house_cards_links))
house_cards_links = [house for house in house_cards_links if not "export" in house]
house_cards_links[:25]

['https://sochi.cian.ru/sale/suburban/300698682/',
 'https://sochi.cian.ru/sale/suburban/273394718/',
 'https://sochi.cian.ru/sale/suburban/278055569/',
 'https://sochi.cian.ru/sale/suburban/273397738/',
 'https://sochi.cian.ru/sale/suburban/295676027/',
 'https://sochi.cian.ru/sale/suburban/302122042/',
 'https://sochi.cian.ru/sale/suburban/273425212/',
 'https://sochi.cian.ru/sale/suburban/300651239/',
 'https://sochi.cian.ru/sale/suburban/298265456/',
 'https://sochi.cian.ru/sale/suburban/294290938/',
 'https://sochi.cian.ru/sale/suburban/283699307/',
 'https://sochi.cian.ru/sale/suburban/255474823/',
 'https://sochi.cian.ru/sale/suburban/294290235/',
 'https://sochi.cian.ru/sale/suburban/291694678/',
 'https://sochi.cian.ru/sale/suburban/279551779/',
 'https://sochi.cian.ru/sale/suburban/278898776/',
 'https://sochi.cian.ru/sale/suburban/284972578/',
 'https://sochi.cian.ru/sale/suburban/290315357/',
 'https://sochi.cian.ru/sale/suburban/301822726/',
 'https://sochi.cian.ru/sale/su

In [315]:
len(house_cards_links)

1429

#### The code below parses each listing from the collected links and retrieves house prices and features

In [316]:
from time import sleep

In [458]:
all_houses_list_dicts = []
all_prices_list = []
driver = get_random_useragent_driver()
for i in tqdm(range(len(house_cards_links))):
    if i % 5 == 0 and i > 0:
        driver.quit()
        driver = get_random_useragent_driver()
    driver.get(house_cards_links[i])
    info_tables_texts = driver.find_elements(By.CSS_SELECTOR, "tr[data-name=OfferSummaryInfoItem]")
    sleep(np.random.randint(2))
    price = driver.find_elements(By.CSS_SELECTOR, "div[data-name=PriceInfo]")

    if len(price)==0:
        driver.quit()
        driver = get_random_useragent_driver()
        driver.refresh()
        driver.get(house_cards_links[i])
        sleep(np.random.randint(2, 4))
        price = driver.find_elements(By.CSS_SELECTOR, "div[data-name=PriceInfo]")

    if len(info_tables_texts)==0:
        driver.quit()
        driver = get_random_useragent_driver()
        driver.get(house_cards_links[i])
        sleep(np.random.randint(2, 4))
        driver.refresh()
        info_tables_texts = driver.find_elements(By.CSS_SELECTOR, "tr[data-name=OfferSummaryInfoItem]")
    
    try:
        price = price[0]
        price_text = price.text
    except:
        price_text = "problems_with_price"
    house_features_raw = [x.text for x in info_tables_texts]
    house_features_medium = []
    for f in house_features_raw:
        if "Площадь" in f:
            if " м" in f:
                f_medium = ("Площадь дома", f.split("\n")[1])
                house_features_medium.append(f_medium)
            else:
                f_medium = ("Площадь участка", f.split("\n")[1])
                house_features_medium.append(f_medium)
        elif "Есть" in f:
            f_medium = (f.split("\n")[0], 1)
            house_features_medium.append(f_medium)
        elif "Дополнительно" in f:
            bonus_features = [(x, 1) for x in f.split("\n")[1].split()]
            house_features_medium.extend(bonus_features)
        else:
            house_features_medium.append(tuple(f.split("\n")))
    
    house_features_dict = dict(house_features_medium)
    all_houses_list_dicts.append(house_features_dict)
    all_prices_list.append(price_text)
    


100%|████████████████████████████████████████████████████████████████████████████| 1429/1429 [3:58:40<00:00, 10.02s/it]


In [459]:
all_houses_list_dicts[:1]

[{'Площадь дома': '115 м²',
  'Количество этажей': '3',
  'Площадь участка': '6 сот.',
  'Водоснабжение': 1,
  'Электричество': 1}]

In [460]:
len(all_houses_list_dicts), len(all_prices_list)

(1429, 1429)

In [461]:
all_houses_list_dicts[:5]

[{'Площадь дома': '115 м²',
  'Количество этажей': '3',
  'Площадь участка': '6 сот.',
  'Водоснабжение': 1,
  'Электричество': 1},
 {'Площадь дома': '280 м²',
  'Материал дома': 'Монолитный',
  'Состояние дома': 'Можно жить',
  'Количество этажей': '2',
  'Количество спален': '4',
  'Площадь участка': '10 сот.',
  'Категория земель': 'Земли населённых пунктов',
  'Статус участка': 'Индивидуальное жилищное строительство',
  'Санузел': 'В доме',
  'Канализация': 'Центральная',
  'Водоснабжение': 'Центральное',
  'Отопление': 'Центральное газовое',
  'Электричество': 1,
  'Газ': 'Магистральный в доме или на участке',
  'Погреб': 1,
  'Терраса': 1,
  'Гараж': 1,
  'Баня': 1,
  'Охрана': 1,
  'Бассейн': 1},
 {'Площадь дома': '180 м²',
  'Материал дома': 'Газобетонный блок',
  'Количество этажей': '2',
  'Количество спален': '3',
  'Площадь участка': '6 сот.',
  'Категория земель': 'Земли населённых пунктов',
  'Статус участка': 'Индивидуальное жилищное строительство',
  'Санузел': 'В доме'

In [462]:
all_prices_list[:16], house_cards_links[:16]

(['3 800 000 ₽',
  '25 000 000 ₽',
  '27 500 000 ₽',
  '37 000 000 ₽',
  '85 000 000 ₽',
  '25 000 000 ₽',
  '40 000 000 ₽',
  '32 000 000 ₽',
  '23 500 000 ₽',
  '78 000 000 ₽',
  '15 000 000 ₽',
  '32 000 000 ₽',
  '148 500 000 ₽',
  'problems_with_price',
  'problems_with_price',
  '20 000 000 ₽'],
 ['https://sochi.cian.ru/sale/suburban/300698682/',
  'https://sochi.cian.ru/sale/suburban/273394718/',
  'https://sochi.cian.ru/sale/suburban/278055569/',
  'https://sochi.cian.ru/sale/suburban/273397738/',
  'https://sochi.cian.ru/sale/suburban/295676027/',
  'https://sochi.cian.ru/sale/suburban/302122042/',
  'https://sochi.cian.ru/sale/suburban/273425212/',
  'https://sochi.cian.ru/sale/suburban/300651239/',
  'https://sochi.cian.ru/sale/suburban/298265456/',
  'https://sochi.cian.ru/sale/suburban/294290938/',
  'https://sochi.cian.ru/sale/suburban/283699307/',
  'https://sochi.cian.ru/sale/suburban/255474823/',
  'https://sochi.cian.ru/sale/suburban/294290235/',
  'https://sochi.cian




### Periodically parsing failed due to pop-ups, random banner ads and cookie collection ads, so not every link was able to retrieve all the necessary data. Below are a fraction of ads for which this was possible at once

In [463]:
print(1 - len([x for x in all_prices_list if x=="problems_with_price"]) / len(all_prices_list))

0.7074877536738978


In [464]:
print(1 - len([x for x in all_houses_list_dicts if x=={}]) / len(all_prices_list))

0.8691392582225332


#### And this code iterates over the house references until all the data is collected

In [465]:
while 1 - len([x for x in all_prices_list if x=="problems_with_price"]) / len(all_prices_list) < 1:
    price_skips_indices = []
    for i in tqdm(range(len(all_prices_list))):
        if all_prices_list[i]=="problems_with_price":
            driver.quit()
            driver = get_random_useragent_driver()
            driver.get(house_cards_links[i])
            sleep(np.random.randint(2))
            driver.refresh()
            try:
                price = driver.find_elements(By.CSS_SELECTOR, "div[data-name=PriceInfo]")
                price = price[0].text
                all_prices_list[i] = price
                sleep(5)
            except:
                print(i)
                continue
print(1 - len([x for x in all_prices_list if x=="problems_with_price"]) / len(all_prices_list))

  3%|██▋                                                                             | 49/1429 [02:32<53:19,  2.32s/it]

48


  4%|███▌                                                                            | 64/1429 [03:15<51:05,  2.25s/it]

63


  5%|███▉                                                                            | 71/1429 [03:23<41:17,  1.82s/it]

...

224


100%|██████████████████████████████████████████████████████████████████████████████| 1429/1429 [00:34<00:00, 41.74it/s]


1247


100%|██████████████████████████████████████████████████████████████████████████████| 1429/1429 [00:24<00:00, 58.26it/s]

1.0


In [466]:
while 1 - len([x for x in all_houses_list_dicts if x=={}]) / len(all_prices_list) < 1:
    data_skips_indices = []
    for i in tqdm(range(len(all_houses_list_dicts))):
        if all_houses_list_dicts[i]=={}:
            driver.quit()
            driver = get_random_useragent_driver()
            driver.get(house_cards_links[i])
            sleep(np.random.randint(2))
            driver.refresh()
            try:
                info_tables_texts = driver.find_elements(By.CSS_SELECTOR, "tr[data-name=OfferSummaryInfoItem]")
                house_features_raw = [x.text for x in info_tables_texts]
                house_features_medium = []
                for f in house_features_raw:
                    if "Площадь" in f:
                        if " м" in f:
                            f_medium = ("Площадь дома", f.split("\n")[1])
                            house_features_medium.append(f_medium)
                        else:
                            f_medium = ("Площадь участка", f.split("\n")[1])
                            house_features_medium.append(f_medium)
                    elif "Есть" in f:
                        f_medium = (f.split("\n")[0], 1)
                        house_features_medium.append(f_medium)
                    elif "Дополнительно" in f:
                        bonus_features = [(x, 1) for x in f.split("\n")[1].split()]
                        house_features_medium.extend(bonus_features)
                    else:
                        house_features_medium.append(tuple(f.split("\n")))
                
                house_features_dict = dict(house_features_medium)
                all_houses_list_dicts[i] = house_features_dict
            except:
                print(i)
                continue
print(1 - len([x for x in all_houses_list_dicts if x=={}]) / len(all_prices_list))

100%|█████████████████████████████████████████████████████████████████████████████| 1429/1429 [00:07<00:00, 182.72it/s]

1.0


In [467]:
all_houses_list_dicts

[{'Площадь дома': '115 м²',
  'Количество этажей': '3',
  'Площадь участка': '6 сот.',
  'Водоснабжение': 1,
  'Электричество': 1},
 {'Площадь дома': '280 м²',
  'Материал дома': 'Монолитный',
  'Состояние дома': 'Можно жить',
  'Количество этажей': '2',
  'Количество спален': '4',
  'Площадь участка': '10 сот.',
  'Категория земель': 'Земли населённых пунктов',
  'Статус участка': 'Индивидуальное жилищное строительство',
  'Санузел': 'В доме',
  'Канализация': 'Центральная',
  'Водоснабжение': 'Центральное',
  'Отопление': 'Центральное газовое',
  'Электричество': 1,
  'Газ': 'Магистральный в доме или на участке',
  'Погреб': 1,
  'Терраса': 1,
  'Гараж': 1,
  'Баня': 1,
  'Охрана': 1,
  'Бассейн': 1},
 {'Площадь дома': '180 м²',
  'Материал дома': 'Газобетонный блок',
  'Количество этажей': '2',
  'Количество спален': '3',
  'Площадь участка': '6 сот.',
  'Категория земель': 'Земли населённых пунктов',
  'Статус участка': 'Индивидуальное жилищное строительство',
  'Санузел': 'В доме'

# Basic data cleaning

#### 1) transforming some fake categoricals to numeric data

In [468]:
houses = all_houses_list_dicts.copy()
for h in houses:
    try:
        h['Площадь дома'] = h['Площадь дома'][:-3] if ("м" in h['Площадь дома']) else h['Площадь дома']
        h['Площадь дома'] = float(h['Площадь дома'].replace(',', '.')) if ',' in h['Площадь дома'] else h['Площадь дома']
    except:
        pass
    try:
        h['Площадь участка'] = h['Площадь участка'][:-5] if ("сот" in h['Площадь участка']) else h['Площадь участка']
        h['Площадь участка'] = h['Площадь участка'][:-3]+"00" if ("га" in h['Площадь участка']) else h['Площадь участка']
        h['Площадь участка'] = float(h['Площадь участка'].replace(',', '.')) if ',' in h['Площадь участка'] else h['Площадь участка']
    except: pass
    try: 
        h['Количество этажей'] = int(h['Количество этажей']) 
    except: pass
    try: 
        h['Количество спален'] = int(h['Количество спален']) 
    except: pass

#### 2) filling some obvious nans

In [470]:
import pandas as pd
df = pd.DataFrame(houses)

In [471]:
df['Водоснабжение'] = df['Водоснабжение'].replace({1: 'Есть', np.nan: 'Нет'})
df['Электричество'] = df['Электричество'].replace({1: 'Есть', np.nan: 'Нет'})
df['Санузел'] = df['Санузел'].replace({1: 'Есть', np.nan: 'Не указано'})
df['Канализация'] = df['Канализация'].replace({1: 'Есть', np.nan: 'Нет'})
df['Газ'] = df['Газ'].replace({1: 'Есть', np.nan: 'Нет'})
df['Отопление'] = df['Отопление'].replace({1: 'Есть', np.nan: 'Нет'})
df['Материал дома'] = df['Материал дома'].replace({np.nan: 'Другой'})
df['Состояние дома'] = df['Состояние дома'].replace({np.nan: 'Не указано'})
df['Категория земель'] = df['Категория земель'].replace({np.nan: 'Не указано'})
df['Статус участка'] = df['Статус участка'].replace({np.nan: 'Не указано'})
df['Погреб'] = df['Погреб'].fillna(0)
df['Терраса'] = df['Терраса'].fillna(0)
df['Гараж'] = df['Гараж'].fillna(0)
df['Баня'] = df['Баня'].fillna(0)
df['Охрана'] = df['Охрана'].fillna(0)
df['Бассейн'] = df['Бассейн'].fillna(0)
df['Площадь дома'] = df['Площадь дома'].astype(np.float64)
df['Площадь участка'] = df['Площадь участка'].astype(np.float64)
df['Год постройки'] = df['Год постройки'].astype(np.float64)
df['Класс'] = df['Класс'].replace({np.nan: 'Другой'})
df = df.drop(df.columns[-2:], axis=1)
print(df.columns.values)
df.head()

['Площадь дома' 'Количество этажей' 'Площадь участка' 'Водоснабжение'
 'Электричество' 'Материал дома' 'Состояние дома' 'Количество спален'
 'Категория земель' 'Статус участка' 'Санузел' 'Канализация' 'Отопление'
 'Газ' 'Погреб' 'Терраса' 'Гараж' 'Баня' 'Охрана' 'Бассейн'
 'Год постройки' 'Класс']


,Площадь дома,Количество этажей,Площадь участка,Водоснабжение,Электричество,Материал дома,Состояние дома,Количество спален,Категория земель,Статус участка,...,Отопление,Газ,Погреб,Терраса,Гараж,Баня,Охрана,Бассейн,Год постройки,Класс
0,115.0,3.0,6.00,Есть,Есть,Другой,Не указано,NaN,Не указано,Не указано,...,Нет,Нет,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Другой
1,280.0,2.0,10.00,Центральное,Есть,Монолитный,Можно жить,4.0,Земли населённых пунктов,Индивидуальное жилищное строительство,...,Центральное газовое,Магистральный в доме или на участке,1.0,1.0,1.0,1.0,1.0,1.0,NaN,Другой
2,180.0,2.0,6.00,Центральное,Есть,Газобетонный блок,Не указано,3.0,Земли населённых пунктов,Индивидуальное жилищное строительство,...,Центральное газовое,Магистральный в доме или на участке,0.0,1.0,1.0,1.0,1.0,1.0,NaN,Другой
3,400.0,3.0,25.00,Центральное,Есть,Монолитный,Можно жить,4.0,Земли населённых пунктов,Индивидуальное жилищное строительство,...,Центральное газовое,Магистральный в доме или на участке,1.0,1.0,1.0,1.0,1.0,1.0,2015.0,Другой
4,360.0,2.0,4.17,Нет,Нет,Другой,Не указано,NaN,Не указано,Не указано,...,Нет,Нет,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Элитный


In [473]:
prices = [int("".join(x[:-1].split())) for x in all_prices_list]
df["Цена"] = pd.Series(prices, dtype=np.float64)
df.describe()

,Площадь дома,Количество этажей,Площадь участка,Количество спален,Погреб,Терраса,Гараж,Баня,Охрана,Бассейн,Год постройки,Цена
count,1429.000000,1336.000000,1429.000000,1008.000000,1429.000000,1429.000000,1429.000000,1429.000000,1429.000000,1429.000000,629.000000,1.429000e+03
mean,289.958852,2.266467,9.791763,4.425595,0.324003,0.498251,0.649405,0.582225,0.580126,0.648705,2019.251192,4.069980e+07
std,157.104372,0.667680,20.497242,1.488926,0.468165,0.500172,0.477323,0.493365,0.493711,0.477542,5.467960,6.089918e+07
min,30.800000,1.000000,0.300000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1950.000000,3.800000e+06
25%,200.000000,2.000000,5.300000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2019.000000,2.150000e+07
50%,254.000000,2.000000,8.000000,4.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,2020.000000,2.900000e+07
75%,350.000000,3.000000,10.000000,5.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2022.000000,3.800000e+07
max,1500.000000,16.000000,677.000000,20.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2024.000000,1.500000e+09


In [474]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1429 entries, 0 to 1428
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Площадь дома       1429 non-null   float64
 1   Количество этажей  1336 non-null   float64
 2   Площадь участка    1429 non-null   float64
 3   Водоснабжение      1429 non-null   object 
 4   Электричество      1429 non-null   object 
 5   Материал дома      1429 non-null   object 
 6   Состояние дома     1429 non-null   object 
 7   Количество спален  1008 non-null   float64
 8   Категория земель   1429 non-null   object 
 9   Статус участка     1429 non-null   object 
 10  Санузел            1429 non-null   object 
 11  Канализация        1429 non-null   object 
 12  Отопление          1429 non-null   object 
 13  Газ                1429 non-null   object 
 14  Погреб             1429 non-null   float64
 15  Терраса            1429 non-null   float64
 16  Гараж              1429 

In [475]:
categorical_columns = df.select_dtypes(include=['object'])
unique_values = {column: categorical_columns[column].unique() for column in categorical_columns.columns}

for column, values in unique_values.items():
    print(f"{column}: {values}")

Водоснабжение: ['Есть' 'Центральное' 'Нет' 'Скважина']
Электричество: ['Есть' 'Нет']
Материал дома: ['Другой' 'Монолитный' 'Газобетонный блок' 'Блочный' 'Кирпичный'
 'Каркасный' 'Деревянный' 'Пенобетонный блок' 'Газосиликатный блок'
 'Монолитно-кирпичный']
Состояние дома: ['Не указано' 'Можно жить' 'Нужен ремонт' 'Нужно достроить']
Категория земель: ['Не указано' 'Земли населённых пунктов' 'Другое'
 'Земли сельхозназначения']
Статус участка: ['Не указано' 'Индивидуальное жилищное строительство' 'Садоводство'
 'Личное подсобное хозяйство' 'Участок поселений']
Санузел: ['Не указано' 'В доме' '1 в доме' '3 в доме' '2 в доме' '6 в доме'
 '1 на улице' '4 в доме' '5 в доме' '3 на улице' '8 в доме' 'На улице'
 '7 в доме' '10 в доме' '2 на улице' '9 в доме']
Канализация: ['Нет' 'Центральная' 'Есть' 'Септик']
Отопление: ['Нет' 'Центральное газовое' 'Автономное газовое' 'Электрическое' 'Камин'
 'Дизельное' 'Печь' 'Автономное' 'Твердотопливный котел'
 'Центральное угольное']
Газ: ['Нет' 'Магистра

#### 3) translating into English

In [476]:
df_eng = pd.DataFrame()
df_eng['House area'] = df["Площадь дома"]
df_eng['Storeys'] = df["Количество этажей"]
df_eng['Yard Area'] = df["Площадь участка"]
df_eng['Water supply'] = df["Водоснабжение"]
df_eng['Electricity']  = df["Электричество"]
df_eng['House Material'] = df["Материал дома"]
df_eng['House Condition'] = df["Состояние дома"]
df_eng['Number of Bedrooms'] = df["Количество спален"]
df_eng['Land Category'] = df["Категория земель"]
df_eng['Ground Status'] = df["Статус участка"]
df_eng['Bathroom'] = df["Санузел"]
df_eng['Sewerage'] = df["Канализация"]
df_eng['Heating'] = df["Отопление"]
df_eng['Gas'] = df["Газ"]
df_eng['Cellar'] = df["Погреб"]
df_eng['Terrace'] = df["Терраса"]
df_eng['Garage'] = df["Гараж"]
df_eng['Bath'] = df["Баня"]
df_eng['Security'] = df["Охрана"]
df_eng['Swimming pool'] = df["Бассейн"]
df_eng['Year Built'] = df["Год постройки"]
df_eng['Class'] = df["Класс"]
df_eng['Price'] = df["Цена"]
df_eng

,House area,Storeys,Yard Area,Water supply,Electricity,House Material,House Condition,Number of Bedrooms,Land Category,Ground Status,...,Gas,Cellar,Terrace,Garage,Bath,Security,Swimming pool,Year Built,Class,Price
0,115.0,3.0,6.00,Есть,Есть,Другой,Не указано,NaN,Не указано,Не указано,...,Нет,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Другой,3800000.0
1,280.0,2.0,10.00,Центральное,Есть,Монолитный,Можно жить,4.0,Земли населённых пунктов,Индивидуальное жилищное строительство,...,Магистральный в доме или на участке,1.0,1.0,1.0,1.0,1.0,1.0,NaN,Другой,25000000.0
2,180.0,2.0,6.00,Центральное,Есть,Газобетонный блок,Не указано,3.0,Земли населённых пунктов,Индивидуальное жилищное строительство,...,Магистральный в доме или на участке,0.0,1.0,1.0,1.0,1.0,1.0,NaN,Другой,27500000.0
3,400.0,3.0,25.00,Центральное,Есть,Монолитный,Можно жить,4.0,Земли населённых пунктов,Индивидуальное жилищное строительство,...,Магистральный в доме или на участке,1.0,1.0,1.0,1.0,1.0,1.0,2015.0,Другой,37000000.0
4,360.0,2.0,4.17,Нет,Нет,Другой,Не указано,NaN,Не указано,Не указано,...,Нет,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Элитный,85000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1424,285.0,3.0,5.00,Центральное,Есть,Монолитный,Можно жить,4.0,Земли населённых пунктов,Индивидуальное жилищное строительство,...,Магистральный в доме или на участке,0.0,1.0,1.0,1.0,1.0,1.0,2020.0,Другой,35000000.0
1425,260.0,2.0,8.00,Центральное,Есть,Монолитный,Можно жить,4.0,Земли населённых пунктов,Индивидуальное жилищное строительство,...,Магистральный в доме или на участке,0.0,1.0,1.0,1.0,1.0,1.0,NaN,Другой,33000000.0
1426,180.0,NaN,4.00,Есть,Есть,Другой,Не указано,NaN,Не указано,Не указано,...,Есть,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Другой,23000000.0
1427,149.0,2.0,8.00,Центральное,Есть,Монолитный,Можно жить,3.0,Земли населённых пунктов,Индивидуальное жилищное строительство,...,Магистральный в доме или на участке,1.0,1.0,1.0,1.0,1.0,1.0,NaN,Другой,20000000.0


In [477]:
translation_dict = {
    'Есть': 'Yes',
    'Нет': 'No',
    'Центральное': 'Central',
    'Скважина': 'Well',
    'Другой': 'Other',
    'Монолитный': 'Monolithic',
    'Газобетонный блок': 'Gas concrete block',
    'Блочный': 'Block',
    'Кирпичный': 'Brick',
    'Не указано': 'Not specified',
    'Можно жить': 'Can live',
    'Нужен ремонт': 'Needs repairs',
    'Нужно достроить': 'Needs to be completed',
    'Земли населённых пунктов': 'Lands of populated areas',
    'Другое': 'Other',
    'Индивидуальное жилищное строительство': 'Individual housing construction',
    'Садоводство': 'Gardening',
    'Личное подсобное хозяйство': 'Personal subsidiary economy',
    'Участок поселений': 'Settlement plot',
    'В доме': 'In the house',
    'Центральная': 'Central',
    'Септик': 'Septic',
    'Центральное газовое': 'Central gas',
    'Автономное газовое': 'Autonomous gas',
    'Электрическое': 'Electric',
    'Магистральный в доме или на участке': 'Mainline in the house or on the plot',
    'Магистральный по границе': 'Magistralny along the border',
    'Газгольдер': 'Gasholder',
    'Каркасный': 'Frame',
    'Деревянный': 'Wood', 
    'Пенобетонный блок': 'Foam concrete block',
    'Газосиликатный блок': 'Gasosilicate block',
    'Монолитно-кирпичный': 'Monolithic brick',
    'Земли сельхозназначения': 'Agricultural',
    '1 в доме': '1 at home',
    '3 в доме': '3 at home',
    '2 в доме': '2 at home',
    '6 в доме': '6 at home',
    '1 на улице': '1 in yard',
    '4 в доме': '4 at home',
    '5 в доме': '5 at home',
    '3 на улице': '3 in yard',
    '8 в доме': '8 at home',
    'На улице': 'in yard',
    '7 в доме': '7 at home',
    '10 в доме': '10 at home',
    '2 на улице': '2 in yard',
    '9 в доме': '9 at home',
    'Камин': 'Fireplace',
    'Дизельное': 'Diesel',
    'Печь': 'Oven',
    'Автономное': 'Standalone',
    'Твердотопливный котел': 'Solid fuel boiler',
    'Центральное угольное': 'Central coal',
    'Класс': 'Class',
    'Элитный': 'Elite',
    'Комфорт': 'Comfort',
    'Эконом': 'Economy',
    'Бизнес': 'Business'
}

def translate(value):
    return translation_dict.get(value, value)

df_eng = df_eng.applymap(translate)

df_eng.head()

C:\Users\Эдуард\AppData\Local\Temp\ipykernel_55568\223066564.py:66: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_eng = df_eng.applymap(translate)


,House area,Storeys,Yard Area,Water supply,Electricity,House Material,House Condition,Number of Bedrooms,Land Category,Ground Status,...,Gas,Cellar,Terrace,Garage,Bath,Security,Swimming pool,Year Built,Class,Price
0,115.0,3.0,6.00,Yes,Yes,Other,Not specified,NaN,Not specified,Not specified,...,No,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Other,3800000.0
1,280.0,2.0,10.00,Central,Yes,Monolithic,Can live,4.0,Lands of populated areas,Individual housing construction,...,Mainline in the house or on the plot,1.0,1.0,1.0,1.0,1.0,1.0,NaN,Other,25000000.0
2,180.0,2.0,6.00,Central,Yes,Gas concrete block,Not specified,3.0,Lands of populated areas,Individual housing construction,...,Mainline in the house or on the plot,0.0,1.0,1.0,1.0,1.0,1.0,NaN,Other,27500000.0
3,400.0,3.0,25.00,Central,Yes,Monolithic,Can live,4.0,Lands of populated areas,Individual housing construction,...,Mainline in the house or on the plot,1.0,1.0,1.0,1.0,1.0,1.0,2015.0,Other,37000000.0
4,360.0,2.0,4.17,No,No,Other,Not specified,NaN,Not specified,Not specified,...,No,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Elite,85000000.0


In [478]:
df_eng.describe()

,House area,Storeys,Yard Area,Number of Bedrooms,Cellar,Terrace,Garage,Bath,Security,Swimming pool,Year Built,Price
count,1429.000000,1336.000000,1429.000000,1008.000000,1429.000000,1429.000000,1429.000000,1429.000000,1429.000000,1429.000000,629.000000,1.429000e+03
mean,289.958852,2.266467,9.791763,4.425595,0.324003,0.498251,0.649405,0.582225,0.580126,0.648705,2019.251192,4.069980e+07
std,157.104372,0.667680,20.497242,1.488926,0.468165,0.500172,0.477323,0.493365,0.493711,0.477542,5.467960,6.089918e+07
min,30.800000,1.000000,0.300000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1950.000000,3.800000e+06
25%,200.000000,2.000000,5.300000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2019.000000,2.150000e+07
50%,254.000000,2.000000,8.000000,4.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,2020.000000,2.900000e+07
75%,350.000000,3.000000,10.000000,5.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2022.000000,3.800000e+07
max,1500.000000,16.000000,677.000000,20.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2024.000000,1.500000e+09


In [479]:
df_eng.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1429 entries, 0 to 1428
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   House area          1429 non-null   float64
 1   Storeys             1336 non-null   float64
 2   Yard Area           1429 non-null   float64
 3   Water supply        1429 non-null   object 
 4   Electricity         1429 non-null   object 
 5   House Material      1429 non-null   object 
 6   House Condition     1429 non-null   object 
 7   Number of Bedrooms  1008 non-null   float64
 8   Land Category       1429 non-null   object 
 9   Ground Status       1429 non-null   object 
 10  Bathroom            1429 non-null   object 
 11  Sewerage            1429 non-null   object 
 12  Heating             1429 non-null   object 
 13  Gas                 1429 non-null   object 
 14  Cellar              1429 non-null   float64
 15  Terrace             1429 non-null   float64
 16  Garage

In [480]:
categorical_columns = df_eng.select_dtypes(include=['object'])
unique_values = {column: categorical_columns[column].unique() for column in categorical_columns.columns}

for column, values in unique_values.items():
    print(f"{column}: {values}")

Water supply: ['Yes' 'Central' 'No' 'Well']
Electricity: ['Yes' 'No']
House Material: ['Other' 'Monolithic' 'Gas concrete block' 'Block' 'Brick' 'Frame' 'Wood'
 'Foam concrete block' 'Gasosilicate block' 'Monolithic brick']
House Condition: ['Not specified' 'Can live' 'Needs repairs' 'Needs to be completed']
Land Category: ['Not specified' 'Lands of populated areas' 'Other' 'Agricultural']
Ground Status: ['Not specified' 'Individual housing construction' 'Gardening'
 'Personal subsidiary economy' 'Settlement plot']
Bathroom: ['Not specified' 'In the house' '1 at home' '3 at home' '2 at home'
 '6 at home' '1 in yard' '4 at home' '5 at home' '3 in yard' '8 at home'
 'in yard' '7 at home' '10 at home' '2 in yard' '9 at home']
Sewerage: ['No' 'Central' 'Yes' 'Septic']
Heating: ['No' 'Central gas' 'Autonomous gas' 'Electric' 'Fireplace' 'Diesel'
 'Oven' 'Standalone' 'Solid fuel boiler' 'Central coal']
Gas: ['No' 'Mainline in the house or on the plot'
 'Magistralny along the border' 'Yes' 'G

#### 4) removing duplicates and saving to csv

In [483]:
df_eng = df_eng.drop_duplicates()
df_eng.shape

(1408, 23)

In [484]:
df_eng.to_csv('sochi.csv', index=False, encoding='utf-8')